## **Imports**

In [1]:
import math
import numpy as np
import torch
import os
from os.path import join as pjoin
import random
import matplotlib
import pandas as pd
import csv

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation, FFMpegFileWriter
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import mpl_toolkits.mplot3d.axes3d as p3

## **Visualize Function**

In [2]:
kinematic_chain = [[0, 2, 5, 8, 11], 
                       [0, 1, 4, 7, 10], 
                       [0, 3, 6, 9, 12, 15], 
                       [9, 14, 17, 19, 21], 
                       [9, 13, 16, 18, 20]]

def plot_3d_motion(save_path, kinematic_tree, mp_joints, title, figsize=(10, 10), fps=120, radius=4):

    title_sp = title.split(' ')
    if len(title_sp) > 20:
        title = '\n'.join([' '.join(title_sp[:10]), ' '.join(title_sp[10:20]), ' '.join(title_sp[20:])])
    elif len(title_sp) > 10:
        title = '\n'.join([' '.join(title_sp[:10]), ' '.join(title_sp[10:])])

    def init():
        ax.set_xlim3d([-radius / 4, radius / 4])
        ax.set_ylim3d([0, radius / 2])
        ax.set_zlim3d([0, radius / 2])
        fig.suptitle(title, fontsize=20)
        ax.grid(b=False)

    def plot_xzPlane(minx, maxx, miny, minz, maxz):
        ## Plot a plane XZ
        verts = [
            [minx, miny, minz],
            [minx, miny, maxz],
            [maxx, miny, maxz],
            [maxx, miny, minz]
        ]
        xz_plane = Poly3DCollection([verts])
        xz_plane.set_facecolor((0.5, 0.5, 0.5, 0.5))
        ax.add_collection3d(xz_plane)

    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111, projection='3d')
    init()

    mp_data = []
    frame_number = min([data.shape[0] for data in mp_joints])

    colors = ['red', 'green', 'black', 'red', 'blue',
              'darkblue', 'darkblue', 'darkblue', 'darkblue', 'darkblue',
              'darkred', 'darkred', 'darkred', 'darkred', 'darkred']

    mp_offset = list(range(-len(mp_joints)//2, len(mp_joints)//2, 1))
    mp_colors = [[colors[i]] * 15 for i in range(len(mp_offset))]

    for i,joints in enumerate(mp_joints):

        # (seq_len, joints_num, 3)
        data = joints.copy().reshape(len(joints), -1, 3)

        MINS = data.min(axis=0).min(axis=0)
        MAXS = data.max(axis=0).max(axis=0)

        height_offset = MINS[1]
        data[:, :, 1] -= height_offset

        mp_data.append({"joints":data,
                        "MINS":MINS,
                        "MAXS":MAXS,})

    def update(index):
        for line in ax.lines:
            line.remove()
        for collection in ax.collections:
            collection.remove()
        ax.view_init(elev=120, azim=-90)
        # plot_xzPlane(-3, 3, 0, -3, 3)
        for pid,data in enumerate(mp_data):
            for i, (chain, color) in enumerate(zip(kinematic_tree, mp_colors[pid])):
                if i < 5:
                    linewidth = 4.0
                else:
                    linewidth = 2.0
                ax.plot3D(data["joints"][index, chain, 0], data["joints"][index, chain, 1], data["joints"][index, chain, 2], linewidth=linewidth,
                          color=color)

        ax.set_axis_off()
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_zticklabels([])
        
    ani = FuncAnimation(fig, update, frames=frame_number, interval=1000 / fps, repeat=False)
    ani.save(save_path, fps=fps)
    plt.close()
    
    print('done ! animation saved to {}'.format(save_path))

In [5]:
## data directory
data_root = './data/data'
motion_dir = pjoin(data_root, 'motions')
text_dir = pjoin(data_root, 'texts')
path = pjoin(data_root, 'test', '1', 'motion_15.npy')

## pick a random file
fname = os.path.basename(path)

## load motion & text
motion = np.load('data/data/test/16/motion_9.npy')
print('motion shape:', motion.shape)
# with open(pjoin(text_dir, fname.replace('.npy', '.txt'))) as fr:
#     texts = fr.read().strip().split('\n')
    
# print('Number of text descriptions:', len(texts))
# ## pick a random text for visualization
# title = random.choice(texts)
# print('title:', title)

motion shape: (82, 384)


In [6]:
## extract positions for both persons
pos_person_1 = motion[:, :22*3]
pos_person_2 = motion[:, 22*3+21*6:22*3+21*6+22*3]

## save animation
plot_3d_motion('./animation.gif', kinematic_chain, [pos_person_1, pos_person_2], "title", radius=4.0, fps=30)

MovieWriter ffmpeg unavailable; using Pillow instead.


done ! animation saved to ./animation.gif


## **Evaluation Function**

In [31]:
def eval_recall(gt_df, pred_df, verbose=False):

    ks = list(range(1, 11))  # Recall@1 ... Recall@9

    rank_cols = list(pred_df.columns)

    ## convert to numpy
    gt = gt_df["candidate"].values
    preds = pred_df[rank_cols].values

    assert len(gt) == len(preds), "size mismatch"
    
    ## compute Recall@K
    recalls = {k: 0 for k in ks}
    n = len(gt)

    for i in range(n):
        for k in ks:
            if gt[i] in preds[i, :k]:
                recalls[k] += 1

    for k in ks:
        recalls[k] /= n

    ## weighted aggregation
    weights = {k: 1.0 / k for k in ks} # higher weight for smaller k
    weight_sum = sum(weights.values())

    final_score = sum(
        weights[k] * recalls[k] for k in ks
    ) / weight_sum
    
    if verbose:
        for k, val in recalls.items():
            print('recall@{} = {}'.format(k, round(val, 3)))

    return final_score

## **Generate Validation Batches**

In [32]:
def generate_val_batches(data_root, save_dir, batch_size, num_batches):
    
    os.makedirs(save_dir, exist_ok=True)

    with open(os.path.join(data_root, 'train.txt')) as fd:
        train_fnames = fd.read().strip().split('\n') 

    random_fnames = random.sample(train_fnames, k=batch_size * num_batches)
    random_batches = np.array(random_fnames).reshape(num_batches, batch_size)

    gt = [['query_id', 'candidate']]
    for file_idx, batch in enumerate(random_batches, start=1):
        os.makedirs(pjoin(save_dir, str(file_idx)), exist_ok=True)

        rdm_idx = random.randint(0, len(batch)-1)
        random_text_fname = batch[rdm_idx]

        with open(os.path.join(data_root, 'texts', random_text_fname+'.txt')) as fd:
            file_texts = fd.read().strip().split('\n')
            random_text = random.choice(file_texts)

        with open(os.path.join(save_dir, str(file_idx), 'text.txt'), 'w') as fw:
            fw.write(random_text)

        for motion_idx, fname in enumerate(batch, start=1):
            if fname == random_text_fname:
                gt.append([file_idx, motion_idx])

            motion = np.load(os.path.join(data_root, 'motions', fname+'.npy'))

            np.save(pjoin(save_dir, str(file_idx), 'motion_{}.npy'.format(motion_idx)), motion)


    with open(os.path.join(save_dir, 'gt.csv'), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(gt)
        
    print('Done ! val batches saved at {} !'.format(save_dir))

In [33]:
## data dir
data_root = './data/'

## dir to save batches to
save_dir = './data/val/'
    
## number of val batches & number of element per batch
batch_size = 32
num_batches = 30

## generate val batches
generate_val_batches(data_root, save_dir, batch_size, num_batches)

Done ! val batches saved at ./data/val/ !


### **Evaluate random submission**

In [34]:
## generate random submission (30 queries × 10 candidates)
random_sub = np.array([
    np.random.choice(range(1, 33), size=10, replace=False)
    for _ in range(30)
])

## create DataFrame
random_sub_df = pd.DataFrame(
    random_sub,
    index=np.arange(1, 31),
    columns=[f'candidate_{i}' for i in range(1, 11)]
)

random_sub_df.index.name = 'query_id'
random_sub_df.head()

,candidate_1,candidate_2,candidate_3,candidate_4,candidate_5,candidate_6,candidate_7,candidate_8,candidate_9,candidate_10
query_id,,,,,,,,,,
1,27,10,7,6,17,11,2,23,21,24
2,8,21,17,24,12,28,25,23,29,30
3,16,26,17,32,12,4,15,23,9,13
4,11,3,21,20,23,9,16,31,24,7
5,18,12,24,5,32,2,17,8,16,1


In [35]:
## load gt
gt_df = pd.read_csv(pjoin(save_dir, 'gt.csv'))
gt_df.head()

,query_id,candidate
0,1,9
1,2,4
2,3,5
3,4,9
4,5,6


In [36]:
## evaluate random results
racall = eval_recall(gt_df, random_sub_df, verbose=True)
print('Recall score:', racall)

recall@1 = 0.033
recall@2 = 0.1
recall@3 = 0.133
recall@4 = 0.133
recall@5 = 0.2
recall@6 = 0.233
recall@7 = 0.267
recall@8 = 0.367
recall@9 = 0.367
recall@10 = 0.4
Recall score: 0.138161044122296
